In [12]:
import pdfplumber
import re
import pandas as pd
import os
import argparse


In [2]:
def extract_ticket_info(pdf_path):
    data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                departure_location = re.search(r'Hinfahrt:\n(?:Gültig ab: .+)?\n([^,]+),', text)
                arrival_location = re.search(r'Freiburg\(Breisgau\) Hbf', text)
                departure_time = re.search(r'Halt Datum Zeit Gleis Produkte Reservierung\n([^ ]+ \d{2}:\d{2})', text)
                arrival_time = re.search(r'Freiburg\(Breisgau\) Hbf .+ \d{2}:\d{2}', text)
                cost = re.search(r'ICE Fahrkarte .+ (\d+,\d{2}€)', text)
                
                if departure_location and arrival_location and departure_time and arrival_time and cost:
                    data.append({
                        'Departure Location': departure_location.group(1),
                        'Arrival Location': arrival_location.group(0),
                        'Departure Time': departure_time.group(1),
                        'Arrival Time': arrival_time.group(0).split()[-1],
                        'Cost': cost.group(1)
                    })

    df = pd.DataFrame(data)
    return df

In [3]:
pdf_path = 'C:\\Users\\freed\\develop\\DB_OEBB_ticket\\ÖBB\\OEBBTicket-1377389263795587.pdf'
df = extract_ticket_info(pdf_path)

In [7]:
data = []
df = pd.DataFrame(data, columns=['Travel Date', 'departure', 'destination', 'Cost'])
df

,Travel Date,departure,destination,Cost


In [11]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            match = re.search(r'\nHF (\d{2}.\d{2}.\d{2})\nDATUM', text, re.DOTALL)
            if match:
                travel_date = match.group(1)
            
            match = re.search(
                r'DATUM ZEIT VON -> NACH DATUM ZEIT KLASSE\n\d{2}\.\d{2} \d{2}:\d{2} (.*?) -> (.*?) \d{2}\.\d{2} \d{2}:\d{2}',
                text, re.DOTALL)
            if match:
                departure = match.group(1)  
                destination = match.group(2)  
    
            match = re.search(r'\nPREIS EUR (.*?)\nZug', text, re.DOTALL)
            if match:
                cost = match.group(1)
                
df.loc[0] = [travel_date ,departure, destination ,  cost]
df

,Travel Date,departure,destination,Cost
0,18.10.24,SALZBURG HBF,VILLACH HBF,"34,90"


In [6]:
text

'FAHRSCHEIN Wang Yun\n01 ERWACHSENER\nZugbindung\nCIV 1181\nHF 18.10.24\nDATUM ZEIT VON -> NACH DATUM ZEIT KLASSE\n18.10 12:12 SALZBURG HBF -> VILLACH HBF 18.10 14:44 1\n* * * -> * * * *\nÜBER --> <1181> HALLEIN * SPITTAL-MILLSTSEE\n01 Sparschiene Berechtigt nicht zum Vorsteuerabzug\nPREIS EUR 34,90\nZug -> RJ 596\nINKL. UST.:\nKEIN UMTAUSCH / KEINE ERSTATTUNG CRE AT:10% EUR 3,17\n530801\n181024 11:58 9505 03\nWurde bezahlt mittels Kreditkarte mit der Nummer xxxxxxxxxxxx5792.\nHinfahrt: Dauer: 2:32\nHaltestelle Datum Zeit Verkehrsmittel\nZangenabdruck\nSalzburg Hbf 18.10.2024 12:12 RJ 596\nVillach Hbf 18.10.2024 14:44\nIn Österreich akzeptieren wir Ihr PDF-\nTicket digital (Anzeige auf Laptop, Tablet,\nSmartphone). Grenzüberschreitend\ndrucken Sie dieses bitte auf weißem\nPapier A4 Hochformat aus. Wir benötigen\nfür die Kontrolle Ihren Lichtbildausweis.\nBitte behalten Sie Ihr Ticket bis zum\nVerlassen Ihrer Ausstiegsstation.\nEs gilt der Tarif des genutzten\nVerkehrsunternehmens. Tari

In [19]:
def extract_ticket_info_from_folder(folder_path, output_file):
    data = []
    
    # Loop through all PDF files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):  # Process only PDF files
            pdf_path = os.path.join(folder_path, file_name)
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        # Extract travel date
                        travel_date_match = re.search(r'\nHF (\d{2}.\d{2}.\d{2})\nDATUM', text)
                        travel_date = travel_date_match.group(1) if travel_date_match else 'N/A'

                        # Extract train information
                        train_match = re.search(r'DATUM ZEIT VON -> NACH DATUM ZEIT KLASSE\n\d{2}\.\d{2} \d{2}:\d{2} (.*?) -> (.*?) \d{2}\.\d{2} \d{2}:\d{2}', text, re.DOTALL)
                        departure = train_match.group(1).strip() if train_match else 'N/A'
                        destination = train_match.group(2).strip() if train_match else 'N/A'

                        # Extract cost
                        cost_match = re.search(r'\nPREIS EUR (.*?)\nZug', text, re.DOTALL)
                        cost = cost_match.group(1).strip() if cost_match else 'N/A'

                        # Append extracted info to the data list
                        data.append({
                            'Travel Date': travel_date,
                            'Departure': departure,
                            'Destination': destination,
                            'Cost (€)': cost,
                            'File Name': file_name  # Include file name for traceability
                        })

    # Convert the data into a DataFrame
    df = pd.DataFrame(data, columns=['Travel Date', 'Departure','Destination', 'Cost (€)', 'File Name'])

    # Save the DataFrame to an Excel file
    df.to_excel(output_file, index=False)
    return df

In [20]:
cwd = os.getcwd()
cwd

'C:\\Users\\freed\\develop\\DB_OEBB_ticket'

In [21]:
# Example usage
folder_path = 'C:\\Users\\freed\\develop\\DB_OEBB_ticket\\ÖBB'  # Replace with the path to your folder
output_file = 'train_ticket_Oebb.xlsx'  # Replace with your desired output file name
df = extract_ticket_info_from_folder(folder_path, output_file)
